In [ ]:
import os
import pickle
import numpy as np
import math
import pandas as pd
import scipy.signal as signal
import neurokit2 as nk
from scipy.signal import butter, filtfilt

# Basic cleaning for RR intervals
def clean_rr_intervals(rr_intervals, threshold_low=0.3, threshold_high=2.0):
    # Identify valid intervals within the threshold range
    valid_rr_intervals = rr_intervals[(rr_intervals >= threshold_low) & 
                                      (rr_intervals <= threshold_high)]
    
    # Calculate the mean of valid RR intervals
    mean_rr = np.mean(valid_rr_intervals)
    
    # Replace abnormal RR intervals with the mean
    rr_intervals = np.where((rr_intervals >= threshold_low) & 
                            (rr_intervals <= threshold_high),
                            rr_intervals,  # Keep valid intervals as is
                            mean_rr)       # Replace invalid intervals with the mean
    
    return rr_intervals

# Compute heart rate from ECG with R-peak detection
def compute_heart_rate(ecg_signal, sampling_rate):
    ecg_signal = ecg_signal.flatten() if ecg_signal.ndim > 1 else ecg_signal
    peaks, _ = signal.find_peaks(ecg_signal, distance=sampling_rate/2)
    rr_intervals = np.diff(peaks) / sampling_rate
    rr_intervals = clean_rr_intervals(rr_intervals)
    heart_rate = 60 / rr_intervals
    return heart_rate, rr_intervals

# Resample to 4 Hz with interpolation
def resample_to_4hz(values, timestamps, original_rate, target_rate):
    # Create original timestamps based on the sampling rate
    original_timestamps = np.arange(0, math.floor(len(values) / original_rate), 1 / original_rate)
    
    # Make sure the final timestamp matches the length of original timestamps
    new_timestamps = timestamps
    print(len(values), len(original_timestamps))
    
    # Interpolate values to fit the new 4 Hz timestamps
    resampled_values = np.interp(new_timestamps, original_timestamps, values)
    
    return resampled_values


# Calculate SDNN for HRV
def calculate_hrv(rr_intervals):
    return np.std(rr_intervals * 1000)

# Apply lowpass filter for denoising EDA signal
def denoise_eda(eda_signal, fs, cutoff=1):
    nyquist = 0.5 * fs
    b, a = butter(4, cutoff / nyquist, btype='low')
    return filtfilt(b, a, eda_signal)

# Artifact correction for EDA signal
def preprocess_eda(eda_signal, fs=4):
    eda_denoised = denoise_eda(eda_signal, fs)
    return nk.eda_clean(eda_denoised, method="biosppy")


# Paths
folder_path = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\Pkl Files'
output_folder = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\csv Files_cleaned'
os.makedirs(output_folder, exist_ok=True)

# Process each file
for filename in os.listdir(folder_path):
    if filename.endswith('.pkl'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'rb') as file:
            data = pickle.load(file, encoding='latin1')
        
        eda_signal = data['signal']['wrist']['EDA'].flatten()
        ecg_signal = data['signal']['chest']['ECG'].flatten()
        labels = data['label'].flatten()

        fs_ecg = 700
        fs_eda = 4

        timestamps = np.arange(0, len(eda_signal) / fs_eda, 1 / fs_eda)  # 4 Hz timestamps
        eda_preprocessed = preprocess_eda(eda_signal, fs=fs_eda)

        heart_rate, rr_intervals = compute_heart_rate(ecg_signal, fs_ecg)    
        
        resampled_hr = resample_to_4hz(heart_rate, timestamps,len(heart_rate)/timestamps[-1] , fs_eda)
        resampled_rr = resample_to_4hz(rr_intervals,timestamps, len(rr_intervals)/timestamps[-1] , fs_eda)
        
        # Calculate HRV dynamically for each timestamp
        hrv_values = [calculate_hrv(resampled_rr[:np.searchsorted(timestamps, t) + 1]) for t in timestamps]

        # Downsample labels to match EDA (assuming 700 Hz -> 4 Hz)
        labels_downsampled = labels[::175]

        # Ensure matching lengths
        min_length = min(len(eda_preprocessed), len(resampled_hr), len(resampled_rr), len(labels_downsampled), len(hrv_values), len(timestamps))
        
        # Slice each signal to min_length
        eda_preprocessed = eda_preprocessed[:min_length]
        resampled_hr = resampled_hr[:min_length]
        resampled_rr = resampled_rr[:min_length]
        labels_downsampled = labels_downsampled[:min_length]
        timestamps = timestamps[:min_length]
        hrv_values = hrv_values[:min_length]

        # Create and save DataFrame
        df = pd.DataFrame({
            'Time': timestamps,
            'EDA': eda_preprocessed,
            'Heart_Rate': resampled_hr,
            'RR_Interval': resampled_rr,
            'HRV': hrv_values,
            'Label': labels_downsampled
        })

        subject_id = data['subject']
        output_file_path = os.path.join(output_folder, f"subject_{subject_id}.csv")
        df.to_csv(output_file_path, index=False)

        print(f"Saved data for subject {subject_id} to {output_file_path}")


In [21]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, variation, moment
from scipy.signal import find_peaks, welch
from scipy.stats import hmean, gmean, iqr, mode
import neurokit2 as nk
from scipy.integrate import trapezoid

# Statistical Features Function with safe mode calculation
def statistical_features(signal, type=''):
    features = {
        f'{type}_mean': np.mean(signal),
        f'{type}_geometric_mean': gmean(signal) if len(signal) > 0 else np.nan,
        f'{type}_harmonic_mean': hmean(signal) if len(signal) > 0 else np.nan,
        f'{type}_standard_deviation': np.std(signal),
        f'{type}_mean_absolute_deviation': np.mean(np.abs(signal - np.mean(signal))),
        f'{type}_variance': np.var(signal),
        f'{type}_median': np.median(signal),
        f'{type}_maximum': np.max(signal),
        f'{type}_minimum': np.min(signal),
        f'{type}_25th_percentile': np.percentile(signal, 25),
        f'{type}_75th_percentile': np.percentile(signal, 75),
        f'{type}_range': np.max(signal) - np.min(signal),
        f'{type}_interquartile_range': iqr(signal),
        f'{type}_variation_coefficient': variation(signal) if len(signal) > 0 else np.nan,
        f'{type}_central_moment': moment(signal, moment=2),
        f'{type}_skewness': skew(signal) if len(signal) > 0 else np.nan,
        f'{type}_kurtosis': kurtosis(signal) if len(signal) > 0 else np.nan
    }
    return features

# Nonlinear Features Function with safe mode calculation
def nonlinear_features(signal, type=''):
    # Compute approximate entropy and sample entropy
    apen, apen_info = nk.entropy_approximate(signal, delay=1, dimension=2, tolerance='sd')
    sampen, sampen_info = nk.entropy_sample(signal, delay=1, dimension=2, tolerance='sd')
    
    # Store the results in the features dictionary directly
    features = {       
        f'{type}_approx_entropy': float(apen),  # Directly assign apen
        f'{type}_sample_entropy': float(sampen)  # Directly assign sampen
    }

    # Compute SD1 and SD2 manually for Poincaré plot features
    rr_diff = np.diff(signal)
    sd1 = np.sqrt(np.var(rr_diff) / 2)
    sd2 = np.sqrt(2 * np.var(signal) - (np.var(rr_diff) / 2))
    
    features.update({f'{type}_SD1': sd1, f'{type}_SD2': sd2})
    return features



def eda_features(eda_signal, fs=4):
    eda_phasic = nk.eda_phasic(eda_signal, sampling_rate=fs)
    scl = np.mean(eda_phasic['EDA_Tonic'])
    peaks, _ = find_peaks(eda_phasic['EDA_Phasic'], height=0)
    scr_amplitude = np.mean(eda_phasic['EDA_Phasic'][peaks])
    scr_duration = np.mean(np.diff(peaks)) / fs if len(peaks) > 1 else 0
    scr_frequency = len(peaks) / len(eda_signal)
    return {
        'SCL': scl,
        'SCR Amplitude': scr_amplitude,
        'SCR Duration': scr_duration,
        'SCR Frequency': scr_frequency
    }


def hrv_time_domain(rr_intervals, fs=4):
    rr_diff = np.diff(rr_intervals)
    sdnn = np.std(rr_intervals)
    avnn = np.mean(rr_intervals)
    rmssd = np.sqrt(np.mean(rr_diff ** 2))
    nn50 = np.sum(np.abs(rr_diff) > 0.05)
    pnn50 = nn50 / len(rr_diff) * 100
    sdann = np.std([np.std(rr_intervals[i:i + 5 * fs]) for i in range(0, len(rr_intervals), 5 * fs)])
    
    # Cardiorespiratory Arrhythmia Index (RSAindex)
    rsa_index = (np.max(rr_intervals) - np.min(rr_intervals)) / avnn if avnn != 0 else np.nan
    
    # Difference in Deep Breathing (DBD)
    dbd = np.max(rr_intervals) - np.min(rr_intervals)
    
    # Difference Between Exhalation and Inspiration (EI)
    ei_ratio = np.max(rr_intervals) / np.min(rr_intervals) if np.min(rr_intervals) != 0 else np.nan

    return {
        'SDNN': sdnn,
        'SDANN': sdann,
        'AVNN': avnn,
        'RMSSD': rmssd,
        'PNN50': pnn50,
        'RSA Index': rsa_index,
        'Deep Breathing Difference (DBD)': dbd,
        'Exhalation/Inspiration Ratio (EI)': ei_ratio
    }

def hrv_frequency_domain(rr_intervals, fs=4):
    # Using Welch's method to compute frequency-domain features
    f, Pxx = welch(rr_intervals, fs, nperseg=256)
    vlf_band, lf_band, hf_band = (0.003, 0.04), (0.04, 0.15), (0.15, 0.4)
    
    # Replacing trapz with trapezoid
    vlf_power = trapezoid(Pxx[(f >= vlf_band[0]) & (f < vlf_band[1])])
    lf_power = trapezoid(Pxx[(f >= lf_band[0]) & (f < lf_band[1])])
    hf_power = trapezoid(Pxx[(f >= hf_band[0]) & (f < hf_band[1])])
    
    total_power = vlf_power + lf_power + hf_power
    return {
        'Total Power': total_power,
        'VLF Power': vlf_power,
        'LF Power': lf_power,
        'HF Power': hf_power,
        'LF/HF Ratio': lf_power / hf_power if hf_power != 0 else np.nan
    }
    

# Paths and parameters
input_folder = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\csv Files_cleaned'
output_file = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\combined_features_dataset.csv'
sampling_rate = 4
segment_samples = int(5 * 60 * sampling_rate)
overlap_samples = int(4.5 * 60 * sampling_rate)

all_features = []

for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(input_folder, filename))
        
        # Segment data with 5-minute windows and 4-minute overlap
        for start in range(0, len(df) - segment_samples + 1, segment_samples - overlap_samples):
            end = start + segment_samples
            segment = df.iloc[start:end].copy()

            if len(segment) == segment_samples:
                # Count each label in the segment
                label_counts = segment['Label'].value_counts(normalize=True)
                dominant_label = label_counts.idxmax()
                # print(label_counts, dominant_label)
                
                # Extract features for EDA, HRV (RR intervals)
                eda_data = segment['EDA'].values
                rr_data = segment['RR_Interval'].values
                hrv_data = segment['HRV'].values
                
                eda_stats = statistical_features(eda_data,"EDA")
                eda_nonlinear = nonlinear_features(eda_data,"EDA")
                eda_other = eda_features(eda_data, fs=sampling_rate)
                
                rr_stats = statistical_features(rr_data,"HRV")
                rr_nonlinear = nonlinear_features(rr_data,"HRV")
                rr_hrv = {**hrv_time_domain(rr_data), **hrv_frequency_domain(rr_data, fs=sampling_rate)}
                
                # Combine all features and label into one row
                combined_features = {
                    **eda_stats, **eda_nonlinear, **eda_other,
                    **rr_stats, **rr_nonlinear, **rr_hrv,
                    'Label': dominant_label,
                }
                all_features.append(combined_features)

# Create a DataFrame with all extracted features and save
combined_df = pd.DataFrame(all_features)
combined_df.to_csv(output_file, index=False)
print(f"Feature dataset saved to {output_file}")




Feature dataset saved to F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\combined_features_dataset.csv


In [2]:
import pandas as pd

# Paths for input and output files
combined_features_file = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\combined_features_dataset.csv'
eda_only_output_file = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\eda_features_only_dataset.csv'

# Load the previously saved combined features dataset
combined_df = pd.read_csv(combined_features_file)

# Identify and filter columns based on explicit inclusion of EDA-related keywords
eda_keywords = ['EDA', 'SCL', 'SCR Amplitude', 'SCR Duration', 'SCR Frequency', 'Label']
eda_columns = [col for col in combined_df.columns if any(keyword in col for keyword in eda_keywords)]

# Create a DataFrame with only the EDA features
eda_df = combined_df[eda_columns]

# Save the EDA-only features dataset to a new CSV file
eda_df.to_csv(eda_only_output_file, index=False)
print(f"EDA-only feature dataset saved to {eda_only_output_file}")


EDA-only feature dataset saved to F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\eda_features_only_dataset.csv


In [3]:
import pandas as pd

# Paths for input and output files
combined_features_file = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\combined_features_dataset.csv'
core_features_output_file = r'F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\core_features_dataset.csv'

# Load the previously saved combined features dataset
combined_df = pd.read_csv(combined_features_file)

# Identify columns associated with core EDA and HRV features (excluding statistical and nonlinear features)
core_eda_hrv_keywords = [
    'SCL', 'SCR Amplitude', 'SCR Duration', 'SCR Frequency',  # Core EDA features
    'SDNN', 'SDANN', 'AVNN', 'RMSSD', 'PNN50',                # Core HRV time-domain features
    'Total Power', 'VLF Power', 'LF Power', 'HF Power', 'LF/HF Ratio',  # Core HRV frequency-domain features
    'Label'  # Include the label column
]
core_columns = [col for col in combined_df.columns if any(keyword in col for keyword in core_eda_hrv_keywords)]

# Create a DataFrame with only the core EDA and HRV features
core_df = combined_df[core_columns]

# Save the core features dataset to a new CSV file
core_df.to_csv(core_features_output_file, index=False)
print(f"Core EDA and HRV feature dataset saved to {core_features_output_file}")


Core EDA and HRV feature dataset saved to F:\Studies\PainStudies Lab\Stress Assessment\WESAD Dataset\WESAD\core_features_dataset.csv
